<a href="https://colab.research.google.com/github/jleaoneto/PC_eleicoes_2010_2014_2018/blob/Notebooks/Analise_Eleicoes_2010.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Agregando a coluna NR_RECEITA_TOTAL e CD_ELEITO ao arquivo consulta_cand_2014_RN
import pandas as pd
import numpy as np

#Leitura do arquivo
path = '/content/drive/My Drive/Colab Notebooks/receitas_candidatos_2010_RN.csv'
df = pd.read_csv(path, sep=';', encoding='latin1')
#df.info()

#Conversao para float
df['VR_RECEITA'] = df['VR_RECEITA'].str.replace(',','.').astype('float')

#Agregação das receitas por candidato
df_2 = df.groupby(by=['NR_CPF_CANDIDATO'],as_index=False).agg({'VR_RECEITA' : np.sum})
#df_2.reset_index(inplace=True)
#df_2.rename( columns= {'Valor receita' : 'NR_RECEITA_TOTAL'},inplace=True)
#df_2.info()

#Leitura do arquivo consulta_cand_2018_RN e inclusão da coluna
path_2 = '/content/drive/My Drive/Colab Notebooks/consulta_cand_2010_RN.csv'
df_3 = pd.read_csv(path_2, sep=';', encoding='latin1')

df_3 = df_3.query('DS_CARGO == "DEPUTADO ESTADUAL" or DS_CARGO == "DEPUTADO FEDERAL"')[['ANO_ELEICAO','DS_CARGO','NR_CPF_CANDIDATO','DS_GENERO','DS_SIT_TOT_TURNO']]

df_3 = df_3.merge(df_2, on = 'NR_CPF_CANDIDATO', how='left')

#Função para criar coluna CD_ELEITO (boolean) com base na colunca DS_SIT_TOT_TURNO
def label_eleito (row):
  if row['DS_SIT_TOT_TURNO'] == "ELEITO" :
    return 1
  elif row['DS_SIT_TOT_TURNO'] == "MÉDIA" :
    return 1
  else :
    return 0

#Criação da coluna CD_ELEITO
df_3['CD_ELEITO'] = df_3.apply(lambda row : label_eleito(row), axis = 1)

#Preencher as receitas que não estejam atribuídas
df_3.fillna(value={'VR_RECEITA' : 0}, inplace=True)

#Arredondas o valor para duas casas decimais
df_3['VR_RECEITA'] = df_3['VR_RECEITA'].round(decimals=2)

print(df_3)

df_3.to_csv('/content/drive/My Drive/Colab Notebooks/consulta_cand_2010_RN_com_receita_e_eleito.csv', sep=';', encoding='latin1', index=False)





     ANO_ELEICAO           DS_CARGO  ...  VR_RECEITA CD_ELEITO
0           2010   DEPUTADO FEDERAL  ...   113772.00         0
1           2010   DEPUTADO FEDERAL  ...     2750.00         0
2           2010   DEPUTADO FEDERAL  ...      100.00         0
3           2010   DEPUTADO FEDERAL  ...        0.00         0
4           2010   DEPUTADO FEDERAL  ...    36558.38         0
..           ...                ...  ...         ...       ...
213         2010  DEPUTADO ESTADUAL  ...    20340.08         0
214         2010  DEPUTADO ESTADUAL  ...   185432.79         1
215         2010  DEPUTADO ESTADUAL  ...   780230.00         1
216         2010  DEPUTADO ESTADUAL  ...    11037.08         0
217         2010  DEPUTADO ESTADUAL  ...        0.00         0

[218 rows x 7 columns]
